### SERVQUAL Manual Labeling Interface 
Purpose: Interactive interface for manually labeling SERVQUAL dimensions <br>
target: 800 reviews from dataset creation <br>
Estimated time: 8-10 hours  

In [18]:
import sys
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt

# Add project root to path
sys.path.append('C:/Users/User/Desktop/ABSA project')

# File paths
training_dir = "C:/Users/User/Desktop/ABSA project/data/training"
input_file = os.path.join(training_dir, "servqual_unlabeled_dataset.json")
output_file = os.path.join(training_dir, "servqual_labeled_dataset.json")
progress_file = os.path.join(training_dir, "labeling_progress.json")

print("Manual SERVQUAL Labeling Interface")
print(f"Input file: {input_file}")
print(f"Output file: {output_file}")
print(f"Progress file: {progress_file}")

Manual SERVQUAL Labeling Interface
Input file: C:/Users/User/Desktop/ABSA project/data/training\servqual_unlabeled_dataset.json
Output file: C:/Users/User/Desktop/ABSA project/data/training\servqual_labeled_dataset.json
Progress file: C:/Users/User/Desktop/ABSA project/data/training\labeling_progress.json


In [11]:
# Load the unlabeled dataset
print("📂 Loading dataset...")
with open(input_file, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

print(f"✅ Loaded {len(dataset)} reviews for labeling")

# Initialize or load progress
if os.path.exists(progress_file):
    print("📊 Loading existing progress...")
    with open(progress_file, 'r', encoding='utf-8') as f:
        progress_data = json.load(f)
    current_index = progress_data.get('current_index', 0)
    completed_count = progress_data.get('completed_count', 0)
    print(f"📈 Resuming from review #{current_index + 1} ({completed_count} completed)")
else:
    print("🆕 Starting new labeling session...")
    current_index = 0
    completed_count = 0
    progress_data = {
        'current_index': 0,
        'completed_count': 0,
        'start_time': datetime.now().isoformat(),
        'last_saved': None
    }

# Load existing labeled data if available
if os.path.exists(output_file):
    print("📋 Loading existing labeled data...")
    with open(output_file, 'r', encoding='utf-8') as f:
        labeled_data = json.load(f)
    
    # Merge with current dataset
    labeled_dict = {item['review_id']: item for item in labeled_data}
    for i, review in enumerate(dataset):
        if review['review_id'] in labeled_dict:
            dataset[i] = labeled_dict[review['review_id']]
            if dataset[i]['labeling_status'] == 'completed':
                completed_count += 1
else:
    labeled_data = []

print(f"🎯 Progress: {completed_count}/{len(dataset)} reviews completed ({completed_count/len(dataset)*100:.1f}%)")

📂 Loading dataset...
✅ Loaded 800 reviews for labeling
🆕 Starting new labeling session...
🎯 Progress: 0/800 reviews completed (0.0%)


In [12]:
class ServqualLabelingInterface:
    def __init__(self, dataset, current_index=0):
        self.dataset = dataset
        self.current_index = current_index
        self.total_reviews = len(dataset)
        
        # Create widgets
        self.create_widgets()
        self.setup_layout()
        
        # Load current review
        self.load_review()
    
    def create_widgets(self):
        """Create all interface widgets."""
        
        # Navigation widgets
        self.progress_label = widgets.HTML()
        self.review_info = widgets.HTML()
        self.review_content = widgets.HTML()
        
        # SERVQUAL dimension widgets
        self.dimension_widgets = {}
        dimensions = ['reliability', 'assurance', 'tangibles', 'empathy', 'responsiveness']
        
        for dim in dimensions:
            self.dimension_widgets[dim] = {
                'relevant': widgets.Checkbox(
                    value=False,
                    description=f'{dim.title()} Relevant',
                    style={'description_width': '150px'},
                    layout=widgets.Layout(width='200px')
                ),
                'sentiment': widgets.FloatSlider(
                    value=0.0,
                    min=-1.0,
                    max=1.0,
                    step=0.1,
                    description=f'{dim.title()} Sentiment',
                    style={'description_width': '150px'},
                    layout=widgets.Layout(width='400px'),
                    readout_format='.1f'
                )
            }
        
        # Notes and navigation
        self.notes_field = widgets.Textarea(
            value='',
            placeholder='Add any notes about this review...',
            description='Notes:',
            layout=widgets.Layout(width='600px', height='80px')
        )
        
        # Navigation buttons
        self.prev_button = widgets.Button(
            description='◀ Previous',
            button_style='info',
            layout=widgets.Layout(width='100px')
        )
        
        self.next_button = widgets.Button(
            description='Next ▶',
            button_style='success',
            layout=widgets.Layout(width='100px')
        )
        
        self.save_button = widgets.Button(
            description='💾 Save',
            button_style='warning',
            layout=widgets.Layout(width='100px')
        )
        
        self.jump_input = widgets.IntText(
            value=1,
            description='Jump to:',
            layout=widgets.Layout(width='150px')
        )
        
        self.jump_button = widgets.Button(
            description='Go',
            layout=widgets.Layout(width='50px')
        )
        
        # Bind events
        self.prev_button.on_click(self.on_previous)
        self.next_button.on_click(self.on_next)
        self.save_button.on_click(self.on_save)
        self.jump_button.on_click(self.on_jump)
    
    def setup_layout(self):
        """Setup the widget layout."""
        
        # Header section
        header = widgets.VBox([
            self.progress_label,
            self.review_info,
            widgets.HTML("<hr>"),
            self.review_content,
            widgets.HTML("<hr>")
        ])
        
        # SERVQUAL labeling section
        dimension_boxes = []
        for dim, widgets_dict in self.dimension_widgets.items():
            dim_box = widgets.HBox([
                widgets_dict['relevant'],
                widgets_dict['sentiment']
            ])
            dimension_boxes.append(dim_box)
        
        servqual_section = widgets.VBox([
            widgets.HTML("""<div style="background-color: #e3f2fd; padding: 10px; border-radius: 5px; margin: 10px 0;">
                           <h3 style="color: #1976d2; margin: 0;">🎯 SERVQUAL Dimension Labeling</h3>
                           </div>"""),
            *dimension_boxes
        ])
        
        # Notes section
        notes_section = widgets.VBox([
            widgets.HTML("""<div style="background-color: #f3e5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">
                           <h3 style="color: #7b1fa2; margin: 0;">📝 Notes</h3>
                           </div>"""),
            self.notes_field
        ])
        
        # Navigation section
        nav_section = widgets.HBox([
            self.prev_button,
            self.next_button,
            self.save_button,
            widgets.HTML("&nbsp;&nbsp;&nbsp;"),
            self.jump_input,
            self.jump_button
        ])
        
        # Complete layout
        self.interface = widgets.VBox([
            header,
            servqual_section,
            notes_section,
            widgets.HTML("<hr>"),
            nav_section
        ])
    
    def load_review(self):
        """Load current review data into interface."""
        if self.current_index >= self.total_reviews:
            self.display_completion()
            return
        
        review = self.dataset[self.current_index]
        
        # Update progress
        completed = sum(1 for r in self.dataset if r['labeling_status'] == 'completed')
        progress_pct = (completed / self.total_reviews) * 100
        
        self.progress_label.value = f"""
        <div style="background-color: #ffffff; color: #333333; padding: 15px; border: 1px solid #ddd; border-radius: 8px; margin-bottom: 10px;">
            <h2 style="color: #2196F3; margin-top: 0;">📊 Review {self.current_index + 1} of {self.total_reviews}</h2>
            <p style="color: #333333;"><strong>Progress:</strong> {completed} completed ({progress_pct:.1f}%)</p>
            <div style="background-color: #e0e0e0; border-radius: 10px; padding: 5px;">
                <div style="background-color: #4CAF50; height: 20px; width: {progress_pct}%; border-radius: 5px;"></div>
            </div>
        </div>
        """
        
        # Update review info
        self.review_info.value = f"""
        <div style="background-color: #f8f9fa; color: #333333; padding: 15px; border: 1px solid #ddd; border-radius: 8px; margin-bottom: 10px;">
            <h3 style="color: #333333; margin-top: 0;">📱 {review['app_name']}</h3>
            <p style="color: #333333;"><strong>Rating:</strong> {'⭐' * review['rating']} ({review['rating']}/5)</p>
            <p style="color: #333333;"><strong>Length:</strong> {review['content_length']} characters</p>
            <p style="color: #333333;"><strong>Date:</strong> {review.get('review_date', 'N/A')}</p>
        </div>
        """
        
        # Update review content
        self.review_content.value = f"""
        <div style="background-color: #f9f9f9; color: #333333; padding: 20px; border-left: 4px solid #2196F3; border: 1px solid #ddd; border-radius: 8px; margin: 10px 0;">
            <h4 style="color: #2196F3; margin-top: 0;">Review Content:</h4>
            <p style="font-size: 16px; line-height: 1.6; color: #333333; background-color: #ffffff; padding: 15px; border-radius: 5px; border: 1px solid #eee;">{review['content']}</p>
        </div>
        """
        
        # Load existing labels if any
        labels = review.get('servqual_labels', {})
        for dim, widgets_dict in self.dimension_widgets.items():
            dim_data = labels.get(dim, {'relevant': False, 'sentiment': 0.0})
            widgets_dict['relevant'].value = dim_data.get('relevant', False)
            widgets_dict['sentiment'].value = dim_data.get('sentiment', 0.0)
        
        # Load notes
        self.notes_field.value = review.get('labeler_notes', '')
        
        # Update jump input
        self.jump_input.value = self.current_index + 1
        
        # Update button states
        self.prev_button.disabled = (self.current_index == 0)
        self.next_button.disabled = (self.current_index >= self.total_reviews - 1)
    
    def save_current_review(self):
        """Save current review labels."""
        if self.current_index >= self.total_reviews:
            return
        
        review = self.dataset[self.current_index]
        
        # Collect SERVQUAL labels
        servqual_labels = {}
        for dim, widgets_dict in self.dimension_widgets.items():
            servqual_labels[dim] = {
                'relevant': widgets_dict['relevant'].value,
                'sentiment': float(widgets_dict['sentiment'].value)
            }
        
        # Update review data
        review['servqual_labels'] = servqual_labels
        review['labeler_notes'] = self.notes_field.value
        review['labeling_status'] = 'completed'
        review['labeled_at'] = datetime.now().isoformat()
        
        print(f"💾 Saved review {self.current_index + 1}")
    
    def on_previous(self, button):
        """Handle previous button click."""
        if self.current_index > 0:
            self.save_current_review()
            self.current_index -= 1
            self.load_review()
    
    def on_next(self, button):
        """Handle next button click."""
        if self.current_index < self.total_reviews - 1:
            self.save_current_review()
            self.current_index += 1
            self.load_review()
    
    def on_save(self, button):
        """Handle save button click."""
        self.save_current_review()
        self.save_progress()
        print(f"✅ Progress saved! Review {self.current_index + 1} completed.")
    
    def on_jump(self, button):
        """Handle jump button click."""
        target = self.jump_input.value - 1  # Convert to 0-based index
        if 0 <= target < self.total_reviews:
            self.save_current_review()
            self.current_index = target
            self.load_review()
    
    def save_progress(self):
        """Save progress and labeled data to files."""
        # Save current progress
        completed = sum(1 for r in self.dataset if r['labeling_status'] == 'completed')
        progress_data = {
            'current_index': self.current_index,
            'completed_count': completed,
            'last_saved': datetime.now().isoformat(),
            'total_reviews': self.total_reviews
        }
        
        with open(progress_file, 'w', encoding='utf-8') as f:
            json.dump(progress_data, f, indent=2)
        
        # Save labeled dataset
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.dataset, f, indent=2, ensure_ascii=False)
        
        print(f"📊 Progress: {completed}/{self.total_reviews} reviews completed")
    
    def display_completion(self):
        """Display completion message."""
        completion_html = """
        <div style="text-align: center; padding: 30px; background-color: #4CAF50; color: white; border-radius: 15px; margin: 20px 0; border: 3px solid #45a049;">
            <h1 style="color: white; margin-top: 0;">🎉 LABELING COMPLETE! 🎉</h1>
            <h2 style="color: white;">All reviews have been labeled successfully!</h2>
            <p style="color: white; font-size: 18px;">Ready to proceed to model training phase.</p>
            <p style="color: white; font-size: 16px;">Great job on completing the SERVQUAL labeling!</p>
        </div>
        """
        self.interface = widgets.HTML(completion_html)
    
    def display(self):
        """Display the interface."""
        return self.interface

print("✅ Labeling interface class created")

✅ Labeling interface class created


In [10]:
# Display SERVQUAL dimensions reference
servqual_guide_html = """
<div style="background-color: #ffffff; color: #333333; padding: 20px; border: 2px solid #2196F3; border-radius: 10px; margin-bottom: 20px;">
<h2 style="color: #2196F3; text-align: center;">🎯 SERVQUAL DIMENSIONS REFERENCE GUIDE</h2>

<div style="margin: 15px 0; padding: 15px; background-color: #f8f9fa; border-radius: 8px;">
<h3 style="color: #333333;">📊 RELIABILITY (Product/Service Dependability)</h3>
<p style="color: #333333;">• Product quality, authenticity, durability<br>
• App performance, stability, crashes<br>
• Order accuracy, fulfillment reliability<br>
<strong>Examples:</strong> "app crashes", "fake product", "good quality", "reliable delivery"</p>
</div>

<div style="margin: 15px 0; padding: 15px; background-color: #f8f9fa; border-radius: 8px;">
<h3 style="color: #333333;">🔒 ASSURANCE (Trust, Security, Competence)</h3>
<p style="color: #333333;">• Payment security, fraud protection<br>
• Seller trust, platform credibility<br>
• Customer service knowledge, expertise<br>
<strong>Examples:</strong> "secure payment", "trusted seller", "helpful support", "safe platform"</p>
</div>

<div style="margin: 15px 0; padding: 15px; background-color: #f8f9fa; border-radius: 8px;">
<h3 style="color: #333333;">🎨 TANGIBLES (Interface, Physical Appearance)</h3>
<p style="color: #333333;">• App design, navigation, usability<br>
• Product photos, descriptions, presentation<br>
• Search functionality, checkout process<br>
<strong>Examples:</strong> "easy to use", "good interface", "clear photos", "simple checkout"</p>
</div>

<div style="margin: 15px 0; padding: 15px; background-color: #f8f9fa; border-radius: 8px;">
<h3 style="color: #333333;">❤️ EMPATHY (Personal Attention, Understanding)</h3>
<p style="color: #333333;">• Return policies, customer accommodation<br>
• Personalized service, special requests<br>
• Customer care, policy flexibility<br>
<strong>Examples:</strong> "easy returns", "understood my needs", "personal touch", "flexible policy"</p>
</div>

<div style="margin: 15px 0; padding: 15px; background-color: #f8f9fa; border-radius: 8px;">
<h3 style="color: #333333;">⚡ RESPONSIVENESS (Speed, Timeliness)</h3>
<p style="color: #333333;">• Delivery speed, shipping time<br>
• Customer service response time<br>
• Order tracking, status updates<br>
<strong>Examples:</strong> "fast delivery", "quick support", "good tracking", "prompt response"</p>
</div>

<div style="margin: 15px 0; padding: 15px; background-color: #e3f2fd; border-radius: 8px; border: 1px solid #2196F3;">
<h3 style="color: #1976d2;">💡 LABELING TIPS:</h3>
<p style="color: #333333;">• A review can be relevant to multiple dimensions<br>
• Sentiment scale: -1.0 (very negative) to +1.0 (very positive)<br>
• If unsure, mark as not relevant rather than guessing<br>
• Use notes field for complex cases</p>
</div>
</div>
"""

display(HTML(servqual_guide_html))

In [17]:
# Create and display the labeling interface
print("🚀 Launching SERVQUAL labeling interface...")
print("\n" + "="*60)
print("📋 LABELING INSTRUCTIONS:")
print("1. Read each review carefully")
print("2. Check 'Relevant' for applicable SERVQUAL dimensions")
print("3. Set sentiment score: -1.0 (very negative) to +1.0 (very positive)")
print("4. Add notes for complex cases")
print("5. Click 'Next' to move forward (auto-saves)")
print("6. Use 'Save' button to manually save progress")
print("7. Take breaks - your progress is saved!")
print("="*60)
print()

# Launch interface
labeling_interface = ServqualLabelingInterface(dataset, current_index)

# Display the interface
display(labeling_interface.display())

🚀 Launching SERVQUAL labeling interface...

📋 LABELING INSTRUCTIONS:
1. Read each review carefully
2. Check 'Relevant' for applicable SERVQUAL dimensions
3. Set sentiment score: -1.0 (very negative) to +1.0 (very positive)
4. Add notes for complex cases
5. Click 'Next' to move forward (auto-saves)
6. Use 'Save' button to manually save progress
7. Take breaks - your progress is saved!



In [19]:
def show_labeling_stats():
    """Display current labeling statistics."""
    
    # Count completed reviews
    completed_reviews = [r for r in dataset if r['labeling_status'] == 'completed']
    completed_count = len(completed_reviews)
    total_count = len(dataset)
    
    print(f"📊 LABELING STATISTICS")
    print(f"="*40)
    print(f"Total reviews: {total_count}")
    print(f"Completed: {completed_count}")
    print(f"Remaining: {total_count - completed_count}")
    print(f"Progress: {completed_count/total_count*100:.1f}%")
    
    if completed_count > 0:
        # Analyze dimension distribution
        dimension_stats = {
            'reliability': {'relevant': 0, 'total_sentiment': 0},
            'assurance': {'relevant': 0, 'total_sentiment': 0},
            'tangibles': {'relevant': 0, 'total_sentiment': 0},
            'empathy': {'relevant': 0, 'total_sentiment': 0},
            'responsiveness': {'relevant': 0, 'total_sentiment': 0}
        }
        
        for review in completed_reviews:
            labels = review.get('servqual_labels', {})
            for dim, data in labels.items():
                if data.get('relevant', False):
                    dimension_stats[dim]['relevant'] += 1
                    dimension_stats[dim]['total_sentiment'] += data.get('sentiment', 0)
        
        print(f"\n📈 DIMENSION ANALYSIS:")
        for dim, stats in dimension_stats.items():
            relevant_count = stats['relevant']
            if relevant_count > 0:
                avg_sentiment = stats['total_sentiment'] / relevant_count
                relevance_pct = (relevant_count / completed_count) * 100
                print(f"  {dim.title()}: {relevant_count} relevant ({relevance_pct:.1f}%), avg sentiment: {avg_sentiment:.2f}")
            else:
                print(f"  {dim.title()}: 0 relevant (0.0%)")
    
    return completed_count, total_count

# Show initial stats
completed, total = show_labeling_stats()


📊 LABELING STATISTICS
Total reviews: 800
Completed: 181
Remaining: 619
Progress: 22.6%

📈 DIMENSION ANALYSIS:
  Reliability: 88 relevant (48.6%), avg sentiment: -0.25
  Assurance: 40 relevant (22.1%), avg sentiment: -0.39
  Tangibles: 66 relevant (36.5%), avg sentiment: -0.34
  Empathy: 16 relevant (8.8%), avg sentiment: -0.49
  Responsiveness: 33 relevant (18.2%), avg sentiment: -0.47


In [20]:
def export_progress_report():
    """Export detailed progress report."""
    
    completed_reviews = [r for r in dataset if r['labeling_status'] == 'completed']
    
    # Create detailed report
    report = {
        'export_date': datetime.now().isoformat(),
        'total_reviews': len(dataset),
        'completed_reviews': len(completed_reviews),
        'completion_rate': len(completed_reviews) / len(dataset),
        'app_distribution': {},
        'dimension_analysis': {},
        'quality_metrics': {
            'avg_review_length': 0,
            'rating_distribution': {},
            'notes_added': 0
        }
    }
    
    # Analyze by app
    for review in completed_reviews:
        app_name = review['app_name']
        if app_name not in report['app_distribution']:
            report['app_distribution'][app_name] = 0
        report['app_distribution'][app_name] += 1
    
    # Analyze dimensions
    dimension_stats = {}
    total_sentiment_scores = []
    notes_count = 0
    
    for review in completed_reviews:
        labels = review.get('servqual_labels', {})
        
        # Count notes
        if review.get('labeler_notes', '').strip():
            notes_count += 1
        
        # Analyze dimensions
        for dim, data in labels.items():
            if dim not in dimension_stats:
                dimension_stats[dim] = {'relevant_count': 0, 'sentiments': []}
            
            if data.get('relevant', False):
                dimension_stats[dim]['relevant_count'] += 1
                sentiment = data.get('sentiment', 0)
                dimension_stats[dim]['sentiments'].append(sentiment)
                total_sentiment_scores.append(sentiment)
    
    # Calculate dimension metrics
    for dim, stats in dimension_stats.items():
        relevance_rate = stats['relevant_count'] / len(completed_reviews)
        avg_sentiment = np.mean(stats['sentiments']) if stats['sentiments'] else 0
        sentiment_std = np.std(stats['sentiments']) if stats['sentiments'] else 0
        
        report['dimension_analysis'][dim] = {
            'relevant_count': stats['relevant_count'],
            'relevance_rate': relevance_rate,
            'avg_sentiment': avg_sentiment,
            'sentiment_std': sentiment_std
        }
    
    # Quality metrics
    report['quality_metrics']['notes_added'] = notes_count
    report['quality_metrics']['avg_sentiment_score'] = np.mean(total_sentiment_scores) if total_sentiment_scores else 0
    
    # Save report
    report_file = os.path.join(training_dir, f"labeling_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    with open(report_file, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2)
    
    print(f"📋 Progress report exported: {report_file}")
    return report

def create_backup():
    """Create backup of current progress."""
    backup_file = os.path.join(training_dir, f"servqual_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    with open(backup_file, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, indent=2, ensure_ascii=False)
    print(f"💾 Backup created: {backup_file}")

print("✅ Export and backup functions ready")
print("\n🔧 Available functions:")
print("  • show_labeling_stats() - Display current progress")
print("  • export_progress_report() - Create detailed progress report")
print("  • create_backup() - Backup current progress")

✅ Export and backup functions ready

🔧 Available functions:
  • show_labeling_stats() - Display current progress
  • export_progress_report() - Create detailed progress report
  • create_backup() - Backup current progress


In [21]:
final_instructions = """
🎯 FINAL LABELING TIPS & REMINDERS

⏰ TIME MANAGEMENT:
• Take regular breaks every 50-100 reviews
• Sessions of 1-2 hours work best for quality
• Your progress is automatically saved

🎨 QUALITY GUIDELINES:
• When in doubt, mark as "not relevant" rather than guessing
• Use the notes field for ambiguous cases
• Consistency is more important than perfection
• Reference the guide above when unsure

💡 EFFICIENCY TIPS:
• Use Tab key to navigate between fields quickly
• Most reviews will only be relevant to 1-2 dimensions
• Focus on the main complaint/praise in each review
• Don't overthink - first instinct is usually correct

🎯 TARGET COMPLETION:
• 800 reviews total
• Estimated 8-10 hours total time
• 80-100 reviews per hour is a good pace
• Quality over speed!

📊 READY FOR TRAINING WHEN:
• All 800 reviews completed
• Balanced distribution across dimensions
• Good mix of positive/negative sentiments
• Notes added for complex cases

🚀 Next step after completion: Model Training (Phase 3)
"""

print(final_instructions)


🎯 FINAL LABELING TIPS & REMINDERS

⏰ TIME MANAGEMENT:
• Take regular breaks every 50-100 reviews
• Sessions of 1-2 hours work best for quality
• Your progress is automatically saved

🎨 QUALITY GUIDELINES:
• When in doubt, mark as "not relevant" rather than guessing
• Use the notes field for ambiguous cases
• Consistency is more important than perfection
• Reference the guide above when unsure

💡 EFFICIENCY TIPS:
• Use Tab key to navigate between fields quickly
• Most reviews will only be relevant to 1-2 dimensions
• Focus on the main complaint/praise in each review
• Don't overthink - first instinct is usually correct

🎯 TARGET COMPLETION:
• 800 reviews total
• Estimated 8-10 hours total time
• 80-100 reviews per hour is a good pace
• Quality over speed!

📊 READY FOR TRAINING WHEN:
• All 800 reviews completed
• Balanced distribution across dimensions
• Good mix of positive/negative sentiments
• Notes added for complex cases

🚀 Next step after completion: Model Training (Phase 3)

